In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to see the raw code."></form>''')

In [ ]:

import pandas
from pandas import DataFrame
import re
import io
from ipywidgets import FileUpload

upload = FileUpload(accept='.csv', multiple=False)
upload

In [ ]:
# Load CSV of Poker Now events
if len(upload.data) > 0:
    poker_log: DataFrame = pandas.read_csv(io.BytesIO(upload.data[-1]))
else:
    poker_log: DataFrame = pandas.read_csv('../data/poker_now_log_sample.csv')
    
poker_log.drop(columns='order', inplace=True)
# Reverse events so timestamps are in ascending order
poker_log = poker_log[::-1]
poker_log.reset_index(drop=True, inplace=True)

poker_log = poker_log.assign(hand_number=0, street='')

starting_hand_rxp = re.compile("-- starting hand #(.*)  \(No Limit Texas Hold'em\) \((.*)\) --")

ending_hand_rxp = re.compile("-- ending hand #(\d) --")

# Hand Numbers
curr_idx = 0
while curr_idx < len(poker_log):
    sliced = poker_log[curr_idx:]

    # Find start index and end index of starting/ending hands
    # Since the PokerNow Event Log has showdowns between the end of one hand and the start of the next, we must track
    # both ending hand events and all events before the start of the next hand
    start_and_end = sliced.entry.str.startswith('-- starting hand').nlargest(2)
    # If there is no following

    # Starting Hand
    start_hand_idx = start_and_end.index.values[0]

    # Ending Hand Index
    end_hand_idx = sliced.entry.str.startswith('-- ending hand').idxmax()

    # Next Starting Hand
    if start_and_end.iloc[1]:
        next_start_idx = start_and_end.index.values[1]
    else:
        next_start_idx = len(poker_log)
    # Just before next starting hand
    end_idx = next_start_idx - 1

    street_slice = poker_log[start_hand_idx:end_idx+1]

    if street_slice.entry.str.startswith('Flop:').any():
        flop_idx = street_slice.entry.str.startswith('Flop:').idxmax()
        poker_log.street.iloc[start_hand_idx:flop_idx] = 'preflop'
        if street_slice.entry.str.startswith('Turn:').any():
            turn_idx = street_slice.entry.str.startswith('Turn:').idxmax()
            poker_log.street.iloc[flop_idx:turn_idx] = 'flop'
            if street_slice.entry.str.startswith('River:').any():
                river_idx = street_slice.entry.str.startswith('River:').idxmax()
                poker_log.street.iloc[turn_idx:river_idx] = 'turn'
                poker_log.street.iloc[river_idx:end_hand_idx+1] = 'river'
                if end_idx != end_hand_idx:
                    poker_log.street.iloc[end_hand_idx+1: end_idx+1] = 'showdown'
            else:
                poker_log.street.iloc[turn_idx:end_idx+1] = 'turn'
        else:
            poker_log.street.iloc[flop_idx:end_idx+1] = 'flop'
    else:
        # No flop, skip others
        poker_log.street.iloc[start_hand_idx:end_idx+1] = 'preflop'

    # Parse hand number using regular expression
    start_match = re.match(starting_hand_rxp, sliced.entry[start_hand_idx])
    # Hand number
    hand_number = int(start_match.group(1))
    # Set hand numbers
    # If start of game or showdown occurred last hand, mark hand number appropriately
    # Else mark hand number as it was parsed from the regexp
    poker_log.hand_number.iloc[start_hand_idx:end_idx+1] = hand_number
    curr_idx = next_start_idx

In [ ]:
# poker_log = poker_log[~poker_log['entry'].astype(str).str.startswith('Player stacks: ')]

poker_log = poker_log.assign(player_name='', player_id='', action='', value='')

YOUR_HAND_MATCHER = re.compile(r'(?<=Your hand is ).*')
SHOWS_HAND_MATCHER = re.compile(r'(?<=shows a ).*')

SMALL_BLIND_MATCHER = re.compile('\"(.*)\" posts a small blind of (\d+)(?=\s|$)')
BIG_BLIND_MATCHER = re.compile('\"(.*)\" posts a big blind of (\d+)(?=\s|$)')

BETS_MATCHER = re.compile('\"(.*)\" bets (\d+)[\s\w]*')
RAISES_MATCHER = re.compile('\"(.*)\" raises to (\d+)[\s\w]*')
CALLS_MATCHER = re.compile('\"(.*)\" calls (\d+)[\s\w]*')
FOLDS_MATCHER = re.compile('\"(.*)\" folds$')
CHECKS_MATCHER = re.compile('\"(.*)\" checks$')
COLLECTED_MATCHER = re.compile('\"(.*)\" collected (\d+) from pot')

JOINED_MATCHER = re.compile('The player \"(.*?)\" joined the game with a stack of (\d+)\.')
QUIT_MATCHER = re.compile('The player \"(.*?)\" quits the game with a stack of (\d+)\.')

poker_log.player_name = poker_log.entry.str.extract("\"(.*?) @ .*?\".*")
poker_log.player_id = poker_log.entry.str.extract("\".*? @ (.*?)\".*")
poker_log.player_name.fillna('')
poker_log.player_id.fillna('')
def parse_action_value(entry):
    if SMALL_BLIND_MATCHER.match(entry):
        return 'small_blind', int(SMALL_BLIND_MATCHER.match(entry).group(2))
    elif BIG_BLIND_MATCHER.match(entry):
        return 'big_blind', int(BIG_BLIND_MATCHER.match(entry).group(2))
    elif BETS_MATCHER.match(entry):
        return 'bets', int(BETS_MATCHER.match(entry).group(2))
    elif RAISES_MATCHER.match(entry):
        return 'raises', int(RAISES_MATCHER.match(entry).group(2))
    elif CALLS_MATCHER.match(entry):
        return 'calls', int(CALLS_MATCHER.match(entry).group(2))
    elif CHECKS_MATCHER.match(entry):
        return 'checks', ''
    elif FOLDS_MATCHER.match(entry):
        return 'folds', ''
    elif COLLECTED_MATCHER.match(entry):
        return 'collected', int(COLLECTED_MATCHER.match(entry).group(2))
    elif JOINED_MATCHER.match(entry):
        return 'joined_with_stack', int(JOINED_MATCHER.match(entry).group(2))
    elif QUIT_MATCHER.match(entry):
        return 'quit_with_stack', int(QUIT_MATCHER.match(entry).group(2))
    else:
        return '', ''
poker_log['action'],  poker_log['value'] = zip(*poker_log['entry'].apply(parse_action_value))

In [ ]:
from numpy import nan
from numpy import isnan

player_action_value_df = poker_log.copy(deep=True)
player_action_value_df.replace('', nan, inplace=True)
player_action_value_df.action.replace('joined_with_stack', nan, inplace=True)
player_action_value_df.dropna(subset=['action', 'value'], inplace=True)
player_action_value_df.drop(columns='entry', inplace=True)

"""
First find the winner of the hand (whoever collected)
Then create a dict with the names of the participants per street -> last wager in a street
Merge each dict with the wagers on name
Remove winning name from dictionary.
For each hand, we'll have: a dictionary with everyone who lost money from a person
cody: {antoine: 10, gautam: 10}
row = cody, col = antoine, value

create giant df
for each hand_number
    create dict
    for each street:
        reverse
        for each row:
            add player name to dict with value (max(timestamp)
    add dict to giant df
"""

unique_players = poker_log.player_id.dropna().unique()
unique_player_names = poker_log.player_name.dropna().unique()
name_id_dict = dict(zip(unique_players, unique_player_names))
total_money_won = DataFrame(columns=unique_players, index=unique_players)
total_money_won.fillna(0)
hand_groups = player_action_value_df.groupby('hand_number', sort=False)
for hand_number, hand_group in hand_groups:
    # Multiple winners if split pot
    winners = set(hand_group[hand_group['action'] == 'collected'].player_id.values)
    # print(f'Hand Number: {hand_number}, Winner: {winner_name}')
    loser_dict: dict = {}
    street_groups = hand_group.groupby('street')
    for street, street_group in street_groups:
        player_groups = street_group.groupby('player_id')
        for player, player_group in player_groups:
            if player not in winners:
                # print(f'Player: {player}, Last Play: {player_group.tail(1).value.values[0]} in street: {street}')
                last_play_value = player_group.tail(1).value.values[0]
                if player in loser_dict:
                    loser_dict[player] += last_play_value
                else:
                    loser_dict[player] = last_play_value
    # print(f'Loser Dict: {loser_dict}')
    for loser_name, value in loser_dict.items():
        # print(f'Loser: {loser_name}, Total Lost: {value} in hand: {hand_number}')
        # If split pot, divide by the number of winners
        split_value = value / len(winners)
        for winner_name in winners:
            if not isnan(total_money_won.loc[winner_name, loser_name]):
                total_money_won.loc[winner_name, loser_name] = total_money_won.loc[winner_name, loser_name] + split_value
            else:
                total_money_won.loc[winner_name, loser_name] = split_value
# Ensure indexes and labels are in the same order
total_money_won.sort_index(axis=0, inplace=True)
total_money_won.sort_index(axis=1, inplace=True)
total_money_won.fillna(0, inplace=True)


total_money_difference = total_money_won.copy(deep=True)
for i in range(0, len(total_money_won.columns)):
    for j in range(0, len(total_money_won)):
        # print(f'{i}, {j}')
        total_money_difference.iloc[i, j] = total_money_won.iloc[i, j] - total_money_won.iloc[j, i]

total_money_won.rename(columns=name_id_dict, index=name_id_dict, inplace=True)
total_money_difference.rename(columns=name_id_dict, index=name_id_dict, inplace=True)


## Total Cent Differential

In [ ]:
total_money_difference

## Total Cents Won

In [ ]:
total_money_won

In [ ]:
import plotly.express as px

sot = poker_log[poker_log.entry.str.startswith('Player stacks:')].copy(deep=True)
sot.drop(columns=['street', 'action', 'value'], inplace=True)
joined_quit_log = poker_log[poker_log.action.eq('quit_with_stack') | poker_log.action.eq('joined_with_stack')].copy()
joined_quit_log.drop(columns=['entry','street', 'action'], inplace=True)
joined_quit_log.rename(columns={'value':'stack_size'},inplace=True)

sot.entry = sot.entry.str.lstrip('Player stacks: ')
sot = sot.assign(player_id='', stack_size=0)
sot = sot.assign(split=sot.entry.str.split(' \| ')).explode('split')
STACK_MATCHER = re.compile('#(\d+) \"(.*?) @ (.*?)\" \((\d+)\)$')

def parse_stack_size(entry):
    match = STACK_MATCHER.match(entry)
    return match.group(3), int(match.group(4))

sot['player_id'], sot['stack_size'] = zip(*sot['split'].apply(parse_stack_size))

sot.drop(columns=['entry', 'split'], inplace=True)
sot = sot.append(joined_quit_log)
sot.sort_values(by='at', axis=0, inplace=True)
sot = sot.reset_index(drop=True)

sot.player_name = sot.player_id.map(name_id_dict)

In [ ]:
fig = px.line(sot, x="at", y="stack_size", color="player_name", title ='''Stack Size by Hand''',
        labels={'at':'Time','stack_size':'Chip Amount','player_name':'Player'}, hover_name='stack_size', hover_data=['stack_size'], width=1000, height=600)
fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
         'xanchor': 'center',
        })
fig.update_traces(mode='markers+lines')
fig.show()

In [ ]:
actions = poker_log[poker_log.action.eq('bets') | poker_log.action.eq('calls') | poker_log.action.eq('raises') | poker_log.action.eq('folds') | poker_log.action.eq('checks')].copy()

actions.player_name = actions.player_id.map(name_id_dict)
actions.drop(columns=['entry'], inplace=True)

grouped_actions = actions.groupby(['player_name', 'action']).size().unstack()

px.bar(grouped_actions, title='Total Action Count').show()


In [ ]:
action_breakdown = grouped_actions.apply(lambda x: x/sum(x), axis=1)

action_breakdown_bar_chart = px.bar(action_breakdown, title='Total Action Percentage Breakdown')
action_breakdown_bar_chart.layout.yaxis.tickformat = ',.0%'
action_breakdown_bar_chart.show()

In [ ]:
raise_percent = grouped_actions.drop(columns=['folds']).apply(lambda x: x/sum(x), axis=1)

raise_percent_bar_chart = px.bar(raise_percent.raises, title='# of Raises ÷ (# of Raises + # of Calls + # of Checks + # of Bets)')
raise_percent_bar_chart.layout.yaxis.tickformat = ',.0%'
raise_percent_bar_chart.show()